Restrict the data to documents present in the QREL. That is, for each of the 25 queries only consider documents that have a qrel assessment. You should have about 14193 documents; some of the docIDs will appear multiple times for multiple queries, while most documents will not appear at all. For each query, consider additional non-relevant documents (not from qrel) so that you end up with about 1000 non-relevant docs per query; you can obtain these for example with IR runs from HW1

Split the data randomly into 20 “training” quereis and 5 “testing” queries.

Document-Query IR Features
The general plan is to build a query-doc static feature matrix. 
qid-docid f1 f2 f3 … fd label 
qid-docid f1 f2 f3 … fd label … 
qid-docid f1 f2 f3 … fd label

You can rearrange this matrix in any format required by the training procedure of the learning algorithm.

Extract for each query-doc the IR features. These are your HW1 and HW2 models like BM25, Language Models, Cosine, Proximity, etc. The cells are feature values, or the scores given by the IR functions. The label is the qrel relevance value

In [2]:
import os 
import nltk
import math 
import re
import time
import copy
import pandas as pd
import numpy as np
import random

from elasticsearch import Elasticsearch 
from nltk.stem import PorterStemmer
from collections import OrderedDict
from sklearn import linear_model

global average_doc_length
global total_doc
global vocab_size
global dict_ids
global df_term_freq
global df_ttf_freq
global df_doc_freq
global len_of_doc
global temp_dict_relevance
global training_df, testing_df

In [3]:
## Elasticsearch instance
es = Elasticsearch()

## Initialize stemmer
ps = PorterStemmer()

## Get all the docs from the file location
file_all = r"C:\Users\mm199\IR-hw\HW1_data\AP_DATA\ap89_collection"

## Temove the read me part
list_of_all = os.listdir(file_all)[:-1]

In [4]:
## Read the data from all sources

total_doc = 0
dict_temp = []
doc_id_list_mapping = {}
for doc in list_of_all:
    with open(r"C:\Users\mm199\IR-hw\HW1_data\AP_DATA\ap89_collection\\" + doc, "r") as file_data:
        file_data_read = file_data.read().split("<DOC>")
    for each_file in file_data_read:
        if "</DOCNO>" in each_file:
            doc_id_content = each_file.split("</DOCNO>")
            total_doc += 1
            doc_id = doc_id_content[0].strip("\n<DOCNO>").strip()
            doc_id_list_mapping[total_doc] = doc_id
            doc_content = doc_id_content[1].split("<TEXT>")[1].strip("\n</TEXT>\n</DOC>\n").strip()
            dict_to_iterate = {"docno": doc_id, "text": doc_content}
            dict_temp.append(dict_to_iterate)


In [5]:
## Save the data in elastic search

i = 1
docid_to_docnum = {}
for es_doc in dict_temp:
    docid_to_docnum[es_doc["docno"]] = i
    i += 1
    es.index(index = "ap_dataset", doc_type = "document", id = i, body = es_doc)
    if i % 1000 == 0:
        print ("AT i: ", i)


AT i:  1000
AT i:  2000
AT i:  3000
AT i:  4000
AT i:  5000
AT i:  6000
AT i:  7000
AT i:  8000
AT i:  9000
AT i:  10000
AT i:  11000
AT i:  12000
AT i:  13000
AT i:  14000
AT i:  15000
AT i:  16000
AT i:  17000
AT i:  18000
AT i:  19000
AT i:  20000
AT i:  21000
AT i:  22000
AT i:  23000
AT i:  24000
AT i:  25000
AT i:  26000
AT i:  27000
AT i:  28000
AT i:  29000
AT i:  30000
AT i:  31000
AT i:  32000
AT i:  33000
AT i:  34000
AT i:  35000
AT i:  36000
AT i:  37000
AT i:  38000
AT i:  39000
AT i:  40000
AT i:  41000
AT i:  42000
AT i:  43000
AT i:  44000
AT i:  45000
AT i:  46000
AT i:  47000
AT i:  48000
AT i:  49000
AT i:  50000
AT i:  51000
AT i:  52000
AT i:  53000
AT i:  54000
AT i:  55000
AT i:  56000
AT i:  57000
AT i:  58000
AT i:  59000
AT i:  60000
AT i:  61000
AT i:  62000
AT i:  63000
AT i:  64000
AT i:  65000
AT i:  66000
AT i:  67000
AT i:  68000
AT i:  69000
AT i:  70000
AT i:  71000
AT i:  72000
AT i:  73000
AT i:  74000
AT i:  75000
AT i:  76000
AT i:  77000
AT i:  7

In [6]:
## Read queries

with open(r"C:\Users\mm199\IR-hw\HW1_data\AP_DATA\query_desc.51-100.short.txt", "r") as query:
    query_list = query.readlines()
    
## define the common words from queries
common_words = ['Document', 'discuss', 'exist','determine', 'current','pay','even','taken','type', 'report', 'describe', 'cite', 'include', 'identify', 'make', 'one', 'must', 'second', 'use', 'side', 'take', 'predict']

## get all the words in a query
all_query_in_a_list = {i[:3].strip(".") : nltk.word_tokenize(i[3:]) for i in query_list}

## get stop words from nltk
stop_words = nltk.corpus.stopwords.words("english")

## words of dictionary which are not in stop words or common words
words_list = {}
for key in all_query_in_a_list:
    if all_query_in_a_list[key] != []:
        words_list[key] = [ps.stem(i) for i in all_query_in_a_list[key] if i not in stop_words and i not in common_words and re.match("^\w+",i)]
for key in words_list:
    words_list[key] = [i for i in words_list[key] if i not in stop_words and i not in common_words and re.match("^\w+",i)]

all_unique_words = list(set([word for query_words in words_list.values() for word in query_words]))


In [7]:
## Parse qrel onto a dictionary
def qrel_parser():
    with open(r"C:\Users\mm199\IR-hw\HW5_data\qrels.adhoc.51-100.AP89.txt") as file:
            read_file = file.readlines()
    dictionary_query = {}
    for i in read_file:
        split_i = i.strip("\n").split(" ")
        key = split_i[0]
        if key not in dictionary_query:
            temp_dict = {}
            temp_dict[split_i[2]] = int(split_i[3])
            dictionary_query[key] = temp_dict
        else:
            temp_dict[split_i[2]] = int(split_i[3])
    return dictionary_query

In [8]:
## call the qrel function
dictionary_query = qrel_parser()
print ("Number of docs found: ", sum([len(i) for i in dictionary_query.values()]))

Number of docs found:  14193


In [9]:
## get only 25 query ids and 1000 docs associated to them along with relevance values
temp_dict_relevance = {}

for query in words_list:
    temp_dict_relevance[query] =  copy.deepcopy(dictionary_query[query])
    for docid in docid_to_docnum.keys(): 
        if docid not in dictionary_query[query]:
            temp_dict_relevance[query][docid] = 0
            if len(temp_dict_relevance[query]) == 1000:
                break
    print ("Query completed: ", query)
print ("Completed adding more non relevant docs: ", temp_dict_relevance)

Query completed:  85
Query completed:  59
Query completed:  56
Query completed:  71
Query completed:  64
Query completed:  62
Query completed:  93
Query completed:  99
Query completed:  58
Query completed:  77
Query completed:  54
Query completed:  87
Query completed:  94
Query completed:  100
Query completed:  89
Query completed:  61
Query completed:  95
Query completed:  68
Query completed:  57
Query completed:  97
Query completed:  98
Query completed:  60
Query completed:  80
Query completed:  63
Query completed:  91
Completed adding more non relevant docs:  {'85': {'AP890101-0060': 1, 'AP890104-0221': 0, 'AP890105-0171': 0, 'AP890106-0013': 0, 'AP890107-0037': 1, 'AP890107-0079': 1, 'AP890107-0095': 1, 'AP890107-0129': 1, 'AP890108-0030': 1, 'AP890109-0151': 1, 'AP890111-0037': 1, 'AP890111-0038': 1, 'AP890111-0040': 1, 'AP890111-0042': 1, 'AP890111-0146': 0, 'AP890111-0158': 0, 'AP890112-0180': 1, 'AP890113-0133': 1, 'AP890116-0144': 0, 'AP890116-0163': 1, 'AP890117-0082': 1, 'AP8

In [10]:
## randomly generate train_queries
train_queries = random.sample(list(words_list.keys()),k = 20)

In [14]:
## randomly generate test_queries
test_queries = [i for i in words_list.keys() if i not in train_queries]

In [15]:
test_queries

['56', '62', '61', '80', '91']

In [16]:
## get labels
df_y_label = {}
for key in train_queries:
    for docno in temp_dict_relevance[key]:
        new_keyid = key + "_" + docno
        df_y_label[new_keyid] = temp_dict_relevance[key][docno]

In [18]:
## define and initialize pandas dataframe to save term freq, doc freq and ttf
list_of_values = list(words_list.values())
list_single_item = list(set([i for item in list_of_values for i in item]))

df_term_freq = pd.DataFrame(data = 0, index = range(1,total_doc+1), columns = list_single_item)
df_ttf_freq = pd.DataFrame(data = 0, index = range(1,total_doc+1), columns = list_single_item)
df_doc_freq = pd.DataFrame(data = 0, index = ["doc_freq"], columns = list_single_item)
len_of_doc = pd.DataFrame(data = 0, index = range(1,total_doc+1), columns = ["length"])


In [20]:
## Get total vocab size

body = {
  "size": 0, 
  "aggs": {
    "vocabSize": {
      "cardinality": {
        "field": "text"
      }
    }
  }
}
vocab_res = es.search(index = "ap_dataset", doc_type = "document", body = body)
vocab_size = vocab_res["aggregations"]['vocabSize']['value']

## Get average doc length for any doc id 

doc_id = 1
dict_all = es.termvectors(index = "ap_dataset", doc_type = "document", id = doc_id, term_statistics = True, fields = ["text"])
average_doc_length = dict_all["term_vectors"]["text"]["field_statistics"]["sum_ttf"]/dict_all["term_vectors"]["text"]["field_statistics"]["doc_count"]

## Get all the data and save it in globally defined data frames

def get_terms(word, doc_id):
    len_of_doc_id = 0
    dict_all = es.termvectors(index = "ap_dataset", doc_type = "document", id = doc_id, term_statistics = True, fields = ["text"])
    if dict_all["term_vectors"] != {}:
        temp_dict = dict_all["term_vectors"]["text"]["terms"]
    else:
        return
    for every_word in temp_dict:
        len_of_doc_id += temp_dict[every_word]["term_freq"] 
    len_of_doc["length"][doc_id] = len_of_doc_id   
    try:     
        tfwd = temp_dict[word]["term_freq"]
        ttf = temp_dict[word]["ttf"]
    except KeyError:
        tfwd = 0
        ttf = 0
    df_term_freq[word][doc_id] = tfwd
    df_ttf_freq[word][doc_id] = ttf

## Initialize pandas dataframe with actual term frequency values

for key in words_list:
    t1 = time.time()
    for word in set(words_list[key]):
        for doc_num in temp_dict_relevance[key].keys():
            search_res = es.search(index = "ap_dataset", doc_type = "document",size = 1, body = {"query" : {"match" : {"text" : word}}})
            df_doc_freq[word]["doc_freq"] = search_res["hits"]["total"]
            doc_id = docid_to_docnum[doc_num]
            get_terms(word, doc_id)
    print ("Word completed:", key," time taken : ", (time.time() - t1), " for ids ")

Word completed: 85  time taken :  92.31826782226562  for ids 
Word completed: 59  time taken :  73.44016456604004  for ids 
Word completed: 56  time taken :  49.343979358673096  for ids 
Word completed: 71  time taken :  110.30441164970398  for ids 
Word completed: 64  time taken :  45.277320861816406  for ids 
Word completed: 62  time taken :  62.09014129638672  for ids 
Word completed: 93  time taken :  54.24032998085022  for ids 
Word completed: 99  time taken :  27.112004280090332  for ids 
Word completed: 58  time taken :  35.06657433509827  for ids 
Word completed: 77  time taken :  35.07672882080078  for ids 
Word completed: 54  time taken :  75.33908891677856  for ids 
Word completed: 87  time taken :  67.93903040885925  for ids 
Word completed: 94  time taken :  38.252644777297974  for ids 
Word completed: 100  time taken :  104.444344997406  for ids 
Word completed: 89  time taken :  62.21390414237976  for ids 
Word completed: 61  time taken :  43.30100178718567  for ids 
Wor

In [21]:
## Define all model 

def okapi_model (word_term_freq_list, doc_id):
    okapi = 0.0
    for key in word_term_freq_list:
        tfwd = df_term_freq[key][doc_id]
        okapi += round((tfwd / (tfwd + 0.5 + 1.5 * (len_of_doc["length"][doc_id] / average_doc_length))),4)
    return okapi
    
def tfidf_model(word_term_freq_list, doc_id):  
    tfidf = 0.0
    for key in word_term_freq_list:
        tfwd = df_term_freq[key][doc_id]
        doc_freq = df_doc_freq[key]["doc_freq"]
        if doc_freq == 0:
            tfidf_val = 0
        else:
            tfidf_val = math.log(total_doc / doc_freq)
        tfidf += round(((tfwd / (tfwd + 0.5 + 1.5 * (len_of_doc["length"][doc_id] / average_doc_length))) * tfidf_val),4)
    return tfidf

def okapi_bm25(word_term_freq_list, doc_id):  
    bm25 = 0.0
    tfwq = {}
    k1 = 1.2
    k2 = 500
    b = 0.75
    for key in word_term_freq_list:
        tfwd = df_term_freq[key][doc_id]
        tfwq[key] = word_term_freq_list.count(key)
        bm25 += round(((math.log((total_doc + 0.5) / (df_doc_freq[key]["doc_freq"] + 0.5)))*((tfwd + k1 * tfwd) / (tfwd + k1 * ((1 - b) + b * (len_of_doc["length"][doc_id] / average_doc_length))))*((tfwq[key] + k2 * tfwq[key]) / (tfwq[key] + k2))), 5)
    return bm25 

def laplace_model(word_term_freq_list, doc_id):  
    laplace = 0.0
    for key in word_term_freq_list:
        tfwd = df_term_freq[key][doc_id]
        prob = ((tfwd + 1) / (len_of_doc["length"][doc_id] + vocab_size))
        laplace += math.log(prob)
    return laplace

def jm_model(word_term_freq_list, doc_id):
    jm = 0.0
    epsilon = 0.00000001
    lamda = 0.3
    for key in word_term_freq_list:
        tfwd = df_term_freq[key][doc_id]
        ttf = df_ttf_freq[key][doc_id]
        if tfwd == 0:
            tfwd = epsilon
            ttf = epsilon
        prob_lm = lamda * (tfwd / len_of_doc["length"][doc_id]) + (1 - lamda) * (ttf / vocab_size)
        jm += math.log(prob_lm)
    return jm


In [22]:
## Function for each model per query

def call_each_model(model_name):
    store_result_for_each_query = {}
    for key in words_list:
        result_dict = {}
        for doc_num in temp_dict_relevance[key].keys():
            doc_id = docid_to_docnum[doc_num]
            result_dict[doc_num] = model_name(words_list[key], doc_id)
        store_result_for_each_query[key] = result_dict
        print ("Query completed: ", key)
    return store_result_for_each_query


In [23]:
## Call each model

okapi_result = call_each_model(okapi_model)
tfidf_result = call_each_model(tfidf_model)
okapi_bm25_result = call_each_model(okapi_bm25)
laplace_result = call_each_model(laplace_model)
jm_result = call_each_model(jm_model)



Query completed:  85
Query completed:  59
Query completed:  56
Query completed:  71
Query completed:  64
Query completed:  62
Query completed:  93
Query completed:  99
Query completed:  58
Query completed:  77
Query completed:  54
Query completed:  87
Query completed:  94
Query completed:  100
Query completed:  89
Query completed:  61
Query completed:  95
Query completed:  68
Query completed:  57
Query completed:  97
Query completed:  98
Query completed:  60
Query completed:  80
Query completed:  63
Query completed:  91
Query completed:  85
Query completed:  59
Query completed:  56
Query completed:  71
Query completed:  64
Query completed:  62
Query completed:  93
Query completed:  99
Query completed:  58
Query completed:  77
Query completed:  54
Query completed:  87
Query completed:  94
Query completed:  100
Query completed:  89
Query completed:  61
Query completed:  95
Query completed:  68
Query completed:  57
Query completed:  97
Query completed:  98
Query completed:  60
Query compl

In [24]:
## Combine the indices query num + "_" + docid
def get_indices(queries):
    indices_for_data_frame = []
    for key in queries:
        for docno in okapi_result[key]:
            docno_key = key + "_" + docno
            indices_for_data_frame.append(docno_key)
    return indices_for_data_frame



In [25]:
## Get separate train and test
train_indices = get_indices(train_queries)
test_indices = get_indices(test_queries)


In [28]:
## Map name to model result
all_results = {"okapi":okapi_result, "okapi_tfidf":tfidf_result, "okapi_bm25":okapi_bm25_result, "laplace":laplace_result, "jm":jm_result}
IR_model = list(all_results.keys())

In [29]:
## Create data frame for both test and train
training_df = pd.DataFrame(data = 0.0, columns = IR_model, index = train_indices)
testing_df = pd.DataFrame(data = 0.0, columns = IR_model, index = test_indices)

In [30]:
## Use set value function to actually fill in the data
def add_data(result, model_name, queries,dataframe):
    for key in queries:
        for keyid in result[key]:
            new_keyid = key + "_" + keyid
            dataframe.set_value(new_keyid, model_name, result[key][keyid])
    

In [31]:
## For each model add the result in the dataframe
def populate_df(all_results, queries, dataframe):
    for result in all_results:
        add_data(all_results[result], result, queries, dataframe)
        print ("done", str(result))

In [32]:
## Training and testing data population with model results
populate_df(all_results, train_queries, training_df)
populate_df(all_results, test_queries, testing_df)

done okapi
done okapi_tfidf
done okapi_bm25
done laplace
done jm
done okapi
done okapi_tfidf
done okapi_bm25
done laplace
done jm


In [33]:
## Logistic regression
log_reg = linear_model.LogisticRegression()
fit_data = log_reg.fit(training_df, list(df_y_label.values()))
coeff = (fit_data.coef_)
test_set = testing_df.dot(np.array(fit_data.coef_).T)
train_set = training_df.dot(np.array(fit_data.coef_).T)

In [34]:
## Separate the regression result into dictionary format
def get_score_after_training(queries_list, result_set):
    global temp_dict_relevance
    dictionary_per_query = {}
    for query in queries_list:
        score_dict = {}
        for queryid in temp_dict_relevance[query]:
            new_keyid = query + "_" + queryid
            score_dict[queryid] = result_set[0][new_keyid]
        score_dict = sorted(score_dict.items(), key = lambda x:x[1], reverse=True)
        dictionary_per_query[query] = score_dict
    return dictionary_per_query


In [35]:
## Call save get_score_after_training function to save the result in dictionary
test_result = get_score_after_training(test_queries, test_set)
train_result = get_score_after_training(train_queries, train_set)

In [36]:
## Function to save the result in a file

# <query-number> Q0 <docno> <rank> <score> Exp

def save_result (store_result_for_each_query, file_name):
    with open(file_name,"w") as file:
        for query_num in store_result_for_each_query:
            rank = 1
            for docid, score in store_result_for_each_query[query_num]:
                temp_str = query_num + " Q0 " + docid + " " + str(rank) + " " +  str(score) + " " + "Exp"
                file.write(temp_str)
                file.write("\n")
                rank += 1


In [37]:
## Call save result function to save the data in HDD
save_result (test_result, "testing1.txt")
save_result (train_result, "training1.txt")